In [1]:
import cv2
import os
import numpy as np

def load_images_from_directory(directory, target_size=(100, 100)):
    images = []
    filenames = sorted([f for f in os.listdir(directory) if f.endswith(".jpg") or f.endswith(".png")])
    for filename in filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
            img_resized = cv2.resize(img_rgb, target_size)  
            images.append(img_resized)
    return np.array(images)

In [2]:
train_lr=load_images_from_directory('/kaggle/input/my-div2k-dataset/DIV2K_train_LR_bicubic/X4')
train_hr=load_images_from_directory('/kaggle/input/my-div2k-dataset/DIV2K_train_HR', (400, 400))

In [3]:
import matplotlib.pyplot as plt
print(train_lr.shape, train_hr.shape)
lr=(100, 100, 3)
hr=(400, 400, 3)

(800, 100, 100, 3) (800, 400, 400, 3)


In [4]:
from keras.models import Model
from keras import Layer
from keras.layers import Dense, Conv2D, LeakyReLU, Input, Add, PReLU, BatchNormalization, Flatten
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

2024-08-02 22:29:38.247151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 22:29:38.247283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 22:29:38.383084: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
class MyLayer(Layer):
    def call(self, x):
        return tf.nn.depth_to_space(x, block_size=2)

In [6]:
def build_generator():
    input_layer=Input(shape=lr)
    
    x=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(input_layer)
    x=PReLU()(x)
    
    def add_rrdb(input_tensor):      
        original_input=input_tensor
        
        def add_dense_block(tensor):
            original=tensor
            
            tensor=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(tensor)
            tensor=LeakyReLU(negative_slope=0.2)(tensor)
            
            tensor=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(tensor)
            tensor=LeakyReLU(negative_slope=0.2)(tensor)
            
            tensor=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(tensor)
            tensor=LeakyReLU(negative_slope=0.2)(tensor)
            
            tensor=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(tensor)
            tensor=LeakyReLU(negative_slope=0.2)(tensor)
            
            tensor=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(tensor)
            tensor=Add()([original, tensor])
            
            return tensor
        
        #adding 3 dense blocks
        for i in range(3):
            input_tensor=add_dense_block(input_tensor)
            
        #creating skip connection with input
        input_tensor=Add()([input_tensor, original_input])
        return input_tensor
    
    for i in range(23):
        x=add_rrdb(x)
        
    x=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(x)
    x=MyLayer()(x)
    x=PReLU()(x)
    
    x=Conv2D(filters= 64, kernel_size=(3, 3), strides=(1,1), padding='same')(x)
    x=MyLayer()(x)
    x=PReLU()(x)
    
    x=Conv2D(filters= 3, kernel_size=(3, 3), strides=(1,1), padding='same')(x)
    
    model=Model(inputs=input_layer, outputs=x)
    return model

In [7]:
import matplotlib.pyplot as plt
#img=np.expand_dims(train_lr[0], axis=0)

In [8]:
from tensorflow.keras.applications import VGG19
def build_vgg():
    vgg = VGG19(weights="imagenet", include_top=False, input_shape=hr)
    vgg.trainable = False
    model = Model(inputs=vgg.input, outputs=vgg.get_layer("block5_conv4").output)
    return model

In [9]:
def build_discriminator():
    input_layer=Input(shape=(25, 25, 512))    
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same')(input_layer)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Flatten()(x)
    x = Dense(units=1024)(x)
    x = LeakyReLU(negative_slope=0.2)(x)    
    x = Dense(units=1, activation='sigmoid')(x)    
    model = Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

In [10]:
def combine(g_model, vgg_loss, d_model):
    d_model.trainable=False
    input_layer=Input(shape=lr)
    gen_image=g_model(input_layer)
    gen_image_features=vgg_loss(gen_image)
    validity=d_model(gen_image_features)
    model=Model(inputs=input_layer, outputs=[gen_image_features, validity])
    model.compile(loss=['mse', 'binary_crossentropy'], loss_weights=[1, 1e-3], optimizer=Adam(0.0002, 0.5))
    return model

In [11]:
def train(g_model, d_model, vgg, gan_model, epochs=5, batch_size=100):
    half_batch=int(batch_size/2)
    batches=int(train_lr.shape[0]/batch_size)
    c=0
    for epoch in range(epochs):
        print(f'epoch {epoch+1} started')
        for batch in range(batches):
            indices=np.random.randint(0, train_lr.shape[0], half_batch)
            c+=1
            
            #discriminator training on real samples
            imgs_hr_real=train_hr[indices]
            imgs_hr_real_features=vgg.predict(imgs_hr_real)
            real_lables=np.ones(half_batch)
            d_loss_real=d_model.train_on_batch(imgs_hr_real_features, real_lables)
            
            #training on fake samples
            fake_imgs=g_model.predict(train_lr[indices])
            fake_imgs_features=vgg.predict(fake_imgs)
            fake_lables=np.zeros(half_batch)
            d_loss_fake=d_model.train_on_batch(fake_imgs_features, fake_lables)
            
            #gan training
            idx=np.random.randint(0, train_lr.shape[0], batch_size)
            lr_imgs=train_lr[idx]
            hr_imgs=train_hr[idx]
            hr_imgs_features=vgg.predict(hr_imgs)
            gan_labes=np.ones(batch_size)
            gan_loss=gan_model.train_on_batch(lr_imgs, [hr_imgs_features, gan_labes])
            
            print(f'd_loss on real={d_loss_real} d_loss_on_fake={d_loss_fake} gan_loss={gan_loss}')
            
            if (c%100==0):
                img=np.expand_dims(train_lr[7], axis=0)
                img_hr=g_model.predict(img)
                plt.imshow(img_hr[0])
                plt.show()
                plt.imshow(train_hr[7])
                plt.show()

In [12]:
discriminator=build_discriminator()
generator=build_generator()
vgg=build_vgg()
esrgan=combine(generator, vgg, discriminator)
train(generator, discriminator, vgg, esrgan)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
epoch 1 started
1/2 ━━━━━━━━━━━━━━━━━━━━ 1:28 89s/step

I0000 00:00:1722637882.940528     112 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 ━━━━━━━━━━━━━━━━━━━━ 146s 57s/step


/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
W0000 00:00:1722637949.363747     112 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 28s 13s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 912ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 7s/step 


2024-08-02 22:35:53.444001: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng20{k2=2,k4=1,k5=0,k6=0,k7=0,k19=0} for conv (f32[100,64,400,400]{3,2,1,0}, u8[0]{0}) custom-call(f32[100,64,400,400]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-08-02 22:35:53.668716: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.224942258s
Trying algorithm eng20{k2=2,k4=1,k5=0,k6=0,k7=0,k19=0} for conv (f32[100,64,400,400]{3,2,1,0}, u8[0]{0}) custom-call(f32[100,64,400,400]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 534, in di